In [1]:
# As usual, a bit of setup

import time
import numpy as np
import matplotlib.pyplot as plt
from cs231n.classifiers.fc_net import *
from cs231n.data_utils import get_CIFAR10_data
from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from cs231n.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'


def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [2]:
num_inputs = 2
input_shape = (4, 5, 6)
output_dim = 3

input_size = num_inputs * np.prod(input_shape)
weight_size = output_dim * np.prod(input_shape)

x = np.linspace(-0.1, 0.5, num=input_size).reshape(num_inputs, *input_shape)
w = np.linspace(-0.2, 0.3, num=weight_size).reshape(np.prod(input_shape), output_dim)
b = np.linspace(-0.3, 0.1, num=output_dim)

#out, _ = affine_forward(x, w, b)
correct_out = np.array([[ 1.49834967,  1.70660132,  1.91485297],
                        [ 3.25553199,  3.5141327,   3.77273342]])


In [3]:
N = x.shape[0]
xdim = np.prod(x.shape[1:])
xr = x.reshape((N,xdim))

In [5]:
x.shape

(2, 4, 5, 6)

In [5]:
list(xrange(5, 1, -1))

[5, 4, 3, 2]

In [3]:
?xrange

In [9]:
N, D1, D2, D3 = 200, 50, 60, 3
X = np.random.randn(N, D1)
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)
a = np.maximum(0, X.dot(W1)).dot(W2)

x  = a
gamma = np.asarray([1.0, 2.0, 3.0])
beta = np.asarray([11.0, 12.0, 13.0])



In [14]:
x_mean = x.mean(axis=0)
x_var = x.var(axis=0)
normalized = (x-x_mean)/np.sqrt(x_var)
out = gamma*normalized + beta

(200, 3)

In [2]:
N, D = 100, 500
x = 5 * np.random.randn(N, D) + 12
gamma = np.random.randn(D)
beta = np.random.randn(D)
dout = np.random.randn(N, D)

bn_param = {'mode': 'train'}
out, cache = batchnorm_forward(x, gamma, beta, bn_param)

t1 = time.time()
dx1, dgamma1, dbeta1 = batchnorm_backward(dout, cache)
t2 = time.time()
dx2, dgamma2, dbeta2 = batchnorm_backward_alt(dout, cache)
t3 = time.time()

print 'dx difference: ', rel_error(dx1, dx2)
print 'dgamma difference: ', rel_error(dgamma1, dgamma2)
print 'dbeta difference: ', rel_error(dbeta1, dbeta2)
print 'speedup: %.2fx' % ((t2 - t1) / (t3 - t2))

(100, 500)
dx difference:  1.0
dgamma difference:  0.0
dbeta difference:  0.0
speedup: 0.25x


In [3]:
dgamma = (cache['normalized'] * dout).sum(axis=0)
dbeta = dout.sum(axis=0)
n = dout.shape[0]
#S = cache['x_var']+cache['eps']
varepssqrt = cache["h2"]
shifted_x = cache["h1"]
#partialder = - np.square(cache["x"]-cache["x_mean"]) / (n * np.square(varepssqrt)*varepssqrt)
partialder = shifted_x * shifted_x.sum(axis=0)/ (n * np.square(varepssqrt)*varepssqrt)
print partialder.shape
dx = cache['gamma'] * dout * partialder

(100, 500)


In [18]:
dbeta = dout.sum(axis=0)

#Layer: normalized
dnormalized =  cache['gamma'] * dout

#Layer h1 is the shifted x-es (x-x_mean_)
dh1 = 1./cache['h2'] * dnormalized
#Layer: h2 is the sqrt of the var
dh2 = (-cache['h1']/np.square(cache['h2']) * dnormalized).sum(axis=0)

#Layer: Variance
dvar = 0.5/np.sqrt(cache['x_var']+cache['eps']) * dh2

#Layer: mean. this has arrows to h1 and var, but the local derivative of var wrt mean is zero
dmean = (-1 * dh1).sum(axis=0) #+ 0 * dvar

#Layer; x has arrows to mean, var and h1
n = dout.shape[0]
dx = 1./n * dmean + 1./n*(2*cache['x']-2*cache['x_mean']) * dvar + 1 * dh1

In [15]:
print dh2.shape

(500,)


In [17]:
dvar_alt = -0.5 * cache["gamma"] * dout.dot(cache["h1"])

ValueError: shapes (100,500) and (100,500) not aligned: 500 (dim 1) != 100 (dim 0)